In [ ]:
!pip install pygame

In [ ]:
import pygame as pg
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [ ]:
# Create the bots

# Our main aim is to find the shortest route traversing all planets once.
# Our population will consist of bots with dna containing different routes, 
# ie, some sequence of planets through which we should go.
# eg of DNA : dna = 3,1,2,0

# the fitness will be defined as the total distance travelled by our rocket

class Route():

  def __init__(self, dnaLength):
    self.dnaLength = dnaLength
    self.dna = list()
    self.distance = 0 # fitness 

    # initialize the random DNA for initial population
    # there should be no repetition in dna
    for i in range(self.dnaLength - 1): 
    # use (dnaLength - 1) since the last gene will always be 0 (initial position) 
    # and we wont use genetic algo to optimize that.
        rnd = np.random.randint(1, self.dnaLength) # upper bound excluded
        # to avoid repetition
        while rnd in self.dna:
            rnd = np.random.randint(1, self.dnaLength)
        self.dna.append(rnd)
    self.dna.append(0) # for last position of dna, add 0 manually
    

    # build the crossover method
    # we take sequence of 2 existing/parent dnas and mix them up to create new dna/ offpring
    def mix(self, dna1, dna2):
        # EXAMPLE:
        # dna1 - 1,2,3,4,0
        # dna2 - 4,3,2,1,0
        # dna = dna1.copy()
        # select index to perform change by inheriting gene from dna2
        # i = 1, dna[i] = 2, dna2[i] = 3
        # previous = dna[i]
        # index3 = 2
        # dna[index3] = previous
        # dna = 1,2,2,4,0
        # dna[i] = dna2[i]
        # dna = 1,3,2,4,0
        self.dna = dna1.copy()
        for i in range(self.dnaLength - 1):
          if np.random.rand() <= 0.5: 
          # 50% chance that this condition will be true, therefore, 50% chance that
          # gene will be changed at this index acc to dna2
              previous = self.dna[i]
              inx = self.dna.index(dna2[i])
              self.dna[inx] = previous
              self.dna[i] = dna2[i]

        # First Random partial mutations - mutate the dna slightly so that the offspring 
        # is not too similar to its parents.
        # EXAMPLE : (pseudocode)
        # dna = 1,2,3,4,0
        # i = 1, dna[i] = 2
        # previous = dna[i] = 2
        # rnd = randint(1, self.dnaLength) = 4
        # inx4 = 4
        # dna[inx4] = previous
        # dna = 1,2,3,2,0
        # dna[i] = rnd
        # dna = 1,4,3,2,0

        for i in range(self.dnaLength - 1): # skip last gene
            if np.random.rand() < 0.1: # 10% chance that it will mutate
                previous = self.dna[i]
                rnd = np.random.randint(1, self.dnaLength)
                inx = self.dna.index(rnd)
                self.dna[inx] = previous
                self.dna[i] = rnd

            # Second random partial mutation
            # EXAMPLE : (Pseudocode)
            # dna = 1,2,3,4,5,0
            # i = 2
            # rnd = random(1, self.dnaLength) = 1
            # inx1 = 0
            # insert rnd at index i
            # dna = 1,2,1,3,4,5,0
            # delete dna[inx1]
            # dna = 2,1,3,4,5,0

            # example 2:
            # rnd = 5
            # inx5 = 4
            # insert rnd at index i
            # dna = 1,2,5,3,4,5,0
            # inx5 = 5

            # one planet switches place to some other position in this mutation

            elif np.random.rand() <= 0.1: # again 10% chance of performing this mutation
              rnd = np.random.randint(1, self.dnaLength)
              prevInx = self.dna.index(rnd)
              self.dna.insert(i, rnd)
              if i >= prevInx:
                self.dna.pop(prevInx)
              else:
                self.dna.pop(prevInx + 1)
            

In [ ]:
# initialize the main code
populationSize = 50 # no of bots we have in our population
mutationRate = 0.1 # the chance that a new bot is a complete mutation and not an offspring
nSelected = 5 # no of bots we take at each generation in the selection process, 
# ie, how many parents in total to create new generation

env = Environment() # object of env class
dnaLength =len(env.planets) # no of plnets we have in our system

population = list()

In [ ]:
# create the first random population
for i in range(populationSize):
  route = Route(dnaLength)
  population.append(route)

In [ ]:
# main while loop - will iterate thru every single generation. Inside, evaluate every population, 
# perform selection, crossover and mutation.

generation = 0 # which gen we are in right now
bestDist = np.inf # shortest distance found till now
while True: # infinite loop - will terminate manually
    generation += 1

    # evaluate the population - calculate fitness
    for route in population:
      env.reset()

      for i in range(dnaLength): # iterate thru all genes (planets)
          action = route.dna[i]
          route.distance += env.step(action, 'none') # this function return the distance

      # sort the population
      sortedPop = sorted(population, key = lambda x: x.distance)
      # takes every single object along with its distance, sorts it acc. to distance 
      # and returns the sorted population
      population.clear()

      # check if the best bot in the new population was in fact better
      if sortedPop[0].distance < bestDist:
          bestDist = sortedPop[0].distance

      # adding best previous bots to the population - not an obligatory step
      # preserve genes of some of the best bots from previous generation
      for i in range(nSelected):
          best = sortedPop[i]
          best.distance = 0 # reset bot distance
          population.append(best)
      
      # fill in the rest of the population
      # add a complete mutant and the offsprings of 2 other routes to our population
      left = populationSize - nSelected # how much space we have left in the population
      for i in range(left):
        # create a new object of route class, ie, a new bot to add in popn
        newRoute = Route(dnaLength) # this dna will be completely random
        if np.random.rand() <= mutationRate: # completely random mutation added
          population.append(newRoute)
        else:
          inx1 = np.random.randint(0, nSelected) 
          inx2 = np.random.randint(0, nSelected) # generate 2 random parents
          while inx1 == inx2: # random parents should not be the same
            inx2 = np.random.randint(0, nSelected)
          
          dna1 = sortedPop[inx1].dna
          dna2 = sortedPop[inx2].dna

          newRoute.mix(dna1, dna2) # mix up the 2 dnas
          population.append(newRoute)

      # display the results
      env.reset()
      # display the best route found
      for i in range(dnaLength): # iterate thru the dna
        action = sortedPopulation[0].dna[i] # # get the next planet to go to
        # for the best dna (given by sortedPop[0])
        _ = env.step(action, 'normal') # update the env acc to ith action - returns distance
        # which we dont need

      if generation % 100 == 0: # displaying tye rocket - only for 1 out of every 100 
      # generations
        env.reset()

        for i in range(dnaLength):
          action = sortedPopulation[0].dna[i]
          _ = env.step(action, 'beautiful')
      
      print('Generation: '+ str(generation) + ' Shortest distance: {:.2f}'.format(bestDist) + 'light years')
